In [31]:
import pandas as pd
import os
import re

In [32]:
merged_coords_1 = pd.read_csv("../data/merged_coords_batch1.csv")
merged_coords_2 = pd.read_csv("../data/merged_coords_batch2.csv")
ameriflux_fluxnet_coords = pd.concat(
    [merged_coords_1, merged_coords_2], ignore_index=True
)

In [33]:
stations = pd.read_csv("../data/EC/ICOS/stations.csv")
merged_coords_1 = pd.read_csv("../data/acos_coords_batch1.csv")
merged_coords_2 = pd.read_csv("../data/acos_coords_batch2.csv")
merged_coords = pd.concat([merged_coords_1, merged_coords_2], ignore_index=True)
merged_coords

# Specify the directory where the zip files are located
directory = "/home/hamid/dscovr/data/EC/ICOS/Data"

# Get a list of all zip files in the directory
zip_files = [f for f in os.listdir(directory) if f.endswith(".zip")]

# Extract the names between the first two underscores
names = [re.search("FLX_(.*?)_", f).group(1) for f in zip_files]

stations_set = set(stations["Name"])
merged_coords_set = set(merged_coords["Name"])

# Find the intersection of the two sets
common_names = stations_set & merged_coords_set

# Filter the 'stations' DataFrame to include only the rows with common 'Name'
common_df = stations[stations["Name"].isin(common_names)]

In [34]:
def find_matching_name(id):
    for name in names:
        if name in id:
            return name
    return None


matching_rows = common_df.copy()
matching_rows["ID"] = matching_rows["Id"].apply(find_matching_name)

In [57]:
matching_rows_df = matching_rows[["Name", "ID"]]

# Extract the 'Name', 'Lat', and 'Long' columns from the 'merged_coords' DataFrame
merged_coords_df = merged_coords[["Name", "Lat", "Long"]]

# Merge the two DataFrames on the 'Name' column
result_df = pd.merge(matching_rows_df, merged_coords_df, on="Name")

# Reset the index of the resulting DataFrame
result_df.reset_index(drop=True, inplace=True)
names_df = pd.DataFrame(names, columns=["ID"])
merged_df = pd.merge(names_df, result_df, on="ID")
merged_df.rename(
    columns={"Name": "site_name", "ID": "name", "Long": "Lon"}, inplace=True
)
merged_df

,name,site_name,Lat,Lon
0,FR-Mej,Mejusseaume,48.118400,-1.796350
1,FR-Hes,Hesse,48.674100,7.064650
2,DE-HoH,Hohes Holz,52.086560,11.222350
3,IT-BFt,Bosco Fontana,45.197754,10.741966
4,IT-SR2,San Rossore 2,43.732020,10.290910
...,...,...,...,...
62,DE-Hzd,Hetzdorf,50.963810,13.489780
63,NL-Loo,Loobos,52.166447,5.743550
64,IT-TrF,Torgnon-LD,45.823760,7.560890
65,FI-Sii,Siikaneva,61.832650,24.192850


In [58]:
ameriflux_fluxnet_coords_set = set(ameriflux_fluxnet_coords["name"])
merged_df_set = set(merged_df["name"])

common_names = ameriflux_fluxnet_coords_set & merged_df_set
common_names_list = list(common_names)
len(common_names_list)
# Drop the common names from 'merged_df'
merged_df = merged_df[~merged_df["name"].isin(common_names_list)]

In [59]:
merged_df.to_csv("../data/EC/ICOS/ICOS_fluxnet_coords.csv", index=False)

In [60]:
merged_df

,name,site_name,Lat,Lon
0,FR-Mej,Mejusseaume,48.118400,-1.796350
1,FR-Hes,Hesse,48.674100,7.064650
2,DE-HoH,Hohes Holz,52.086560,11.222350
3,IT-BFt,Bosco Fontana,45.197754,10.741966
8,FR-EM2,Estrees-Mons A28,49.872110,3.020650
9,DE-Har,Hartheim,47.933000,7.598100
13,UK-AMo,Auchencorth Moss,55.792545,-3.243692
14,IT-Lsn,Lison,45.740482,12.750297
18,SE-Htm,Hyltemossa,56.097630,13.418970
19,SE-Htm,Hyltemossa,56.097600,13.418900
